In [1]:
import numpy as np 
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math

In [2]:
#Adding Normalization 
 
inputs = torch.Tensor([[[0.2,0.1,0.3],[0.5,0.1,0.1]]])
B,S,E = inputs.size()
inputs = inputs.reshape(S,B,E)
inputs.size()

torch.Size([2, 1, 3])

In [3]:
parameter_shape = inputs.size()[-2:]
gamma = nn.Parameter(torch.ones(parameter_shape))
beta = nn.Parameter(torch.zeros(parameter_shape))

In [4]:
gamma.size(),beta.size()

(torch.Size([1, 3]), torch.Size([1, 3]))

In [8]:
parameter_shape

torch.Size([1, 3])

In [6]:
dims = [-(i+1) for i in range(len(parameter_shape))]

In [7]:
dims

[-1, -2]

In [9]:
mean = inputs.mean(dim=dims,keepdim=True)

In [11]:
mean,mean.shape

(tensor([[[0.2000]],
 
         [[0.2333]]]),
 torch.Size([2, 1, 1]))

In [13]:
var = ((inputs - mean)** 2).mean(dim = dims,keepdim=True)
epsilon = 1e-5 
std = (var +epsilon).sqrt()
std

tensor([[[0.0817]],

        [[0.1886]]])

In [15]:
y = (inputs- mean)/std 
y

tensor([[[ 0.0000, -1.2238,  1.2238]],

        [[ 1.4140, -0.7070, -0.7070]]])

In [16]:
out = gamma * y +beta

In [17]:
out

tensor([[[ 0.0000, -1.2238,  1.2238]],

        [[ 1.4140, -0.7070, -0.7070]]], grad_fn=<AddBackward0>)

In [23]:
class LayerNormalization():
    def __init__(self,parameter_shape,eps = 1e-5):
        self.parameter_shape = parameter_shape
        self.eps = eps 
        self.gamma = nn.Parameter(torch.ones(parameter_shape))
        self.beta = nn.Parameter(torch.zeros(parameter_shape))
    
    def forward(self,input):
        dims = [-(i+1) for i in range(len(self.parameter_shape))]
        mean = input.mean(dim=dims, keepdim=True)
        var = ((input - mean)** 2).mean(dim = dims, keepdim=True)
        std = (var + self.eps).sqrt()
        y = (input - mean)/std
        out = self.gamma * y + self.beta
        return out

In [24]:
batch_size = 3 
sentence_length =  3
embedding_dim = 8 
inputs = torch.randn(sentence_length,batch_size,embedding_dim)

In [25]:
print(inputs.size)

<built-in method size of Tensor object at 0x0000025A34DA4A90>


In [26]:
layr_norm = LayerNormalization(inputs.size()[-2:])

In [27]:
out = layr_norm.forward(input=inputs)

In [28]:
out

tensor([[[ 1.0415, -0.7700,  2.2907, -0.1778, -0.7716, -0.7125, -0.3898,
          -0.0832],
         [-0.4098,  0.6121,  0.8826,  0.2598,  0.6500, -0.4147, -0.7770,
          -0.8311],
         [ 0.2798, -1.6510, -1.9876, -0.6111,  1.1894,  0.3622,  1.9559,
           0.0634]],

        [[ 1.7726, -1.3757, -1.3025, -0.7694, -0.5503, -1.5530,  0.1295,
          -0.4563],
         [ 0.1407,  1.1382,  0.4874,  1.3179,  0.9057,  0.5360, -1.1850,
          -0.5266],
         [ 0.2174,  0.4561,  1.8645, -1.5164,  0.2167,  0.5882, -1.0063,
           0.4706]],

        [[ 1.6138, -2.3227, -0.0210, -0.0905, -0.7498, -0.1484,  0.4839,
           1.1889],
         [-0.3756,  2.1641,  0.1156, -0.7204,  0.2570,  0.5097, -1.1087,
           0.8703],
         [-0.5434,  1.3544, -0.3705, -0.6061, -1.7984, -0.2249,  0.1287,
           0.3938]]], grad_fn=<AddBackward0>)